In [ ]:
!pip install wikipedia requests BeautifulSoup4 Wikipedia-Api newspaper3k trafilatura lxml_html_clean

In [ ]:
import re

import wikipedia

from bs4 import BeautifulSoup

import requests

import wikipediaapi

import time

from tqdm import tqdm

import trafilatura

from newspaper import Article

import newspaper

from newspaper import fulltext

In [ ]:
article_name = 'Летние_Олимпийские_игры_2024'

In [ ]:
# wikipedia.set_lang("ru")

# page1 = wikipedia.page('Нейронная сеть')

# print(page1.title)



# h = page1.html()



# parser=BeautifulSoup(h, 'html.parser')

# references = parser.find_all("div", attrs={"class":"mw-references-wrap"})

# links_by_citations =[]

# for reference_section in references:

#     links_by_citations.extend((r.get('id'), [(link.get('href'), link) for link in r.find_all("a",attrs={"class":"external text"})]) for r in reference_section.find_all("li"))

# links_by_citations = dict(links_by_citations)

# links_by_citations


In [ ]:
"""wikipedia.set_lang("ru")

page1 = wikipedia.page(article_name)

print(page1.title)"""


headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

response = requests.get('https://ru.ruwiki.ru/wiki/Летние_Олимпийские_игры_2024', headers=headers, stream=True, allow_redirects=False, timeout=(3, 7))

#text = response.text
h = response.text
#h = page1.html()



parser=BeautifulSoup(h, 'html.parser')

references = parser.find_all("div", attrs={"class":"mw-references-wrap"})

links_by_citations = {}



for reference_section in references:

  all_ref =  reference_section.find_all("li")

  for r in all_ref:

    ref_content = []

    ref_id = r.get('id')

    external_links = r.find_all("a", attrs={"class": "external text"})

    if external_links:

      ref_content.extend([(link.get('href'), link.get_text(strip=True)) for link in external_links])

    else:

      reference_text = r.find("span", class_="reference-text")

      if reference_text:

        internal_link = reference_text.find("a", href=True)

        if not internal_link:

          ref_content.append((reference_text.get_text(strip=True)))

        else:

           for ul in parser.find_all("ul"):

              for li in ul.find_all("li"):

                literature = li.find("span", {"class": "citation no-wikidata", "id": internal_link['href'][1:]})

                if literature:

                  ref_content.append((literature.get_text(strip=True)))

    if ref_content:

      links_by_citations[ref_id] = ref_content



links_by_citations

In [ ]:
headers = [[(head.name, head.text) for head in r.find_all(re.compile(r"h[0-9]"))] for r in parser.find_all("div", attrs={"class":"mw-heading"})]

headers

In [ ]:
wiki = wikipediaapi.Wikipedia(

      user_agent='Storm_benchmark',

      language='ru',

      extract_format=wikipediaapi.ExtractFormat.WIKI)

page_api = wiki.page(article_name)

In [ ]:
content = {}

def parse_section(section, level=0):

    if section.text and not section.sections or level == 0:

      if level == 0:

        content[(section.title, "h" + str(level + 1))] = section.text.split('\n\n')[0]

      else:

        content[(section.title, "h" + str(level + 1))] = section.text

    elif section.sections:

       content[(section.title, "h" + str(level + 1))] = ""

    for subsection in section.sections:

        parse_section(subsection, level + 1)



parse_section(page_api)



for key, value in content.items():

  print(key, value)

In [ ]:
link_num = {}

texts = parser.find_all("div", attrs={"class":"mw-parser-output"})

for item, _ in links_by_citations.items():

  for ref in texts:

    all_link =  ref.find_all("sup")

    for link in all_link:

      links_sup = link.find("a")

      if links_sup and links_sup['href'][1:] != item:

        continue

      elif links_sup:

        pattern = r'>\s*(\d+)\s*<'

        pattern2 = r'\d+'

        number = re.search(pattern, str(links_sup)).group(0)

        number2 = re.search(pattern2, str(number))

        link_num[item] = number2.group(0)



link_num

In [ ]:
references_positions = {}

head_tags = set([key[1] for key in content.keys()])

content_div = parser.find('div', {'class': 'mw-parser-output'})

article_text = ''

bias = 0

if content_div:

    for p in content_div.find_all(['p'] + list(head_tags)):

      text = p.get_text()

      for link_name, link in link_num.items():

        pattern = '\[' + str(link) + '\]'

        for match in re.finditer(pattern, text):

            # print(match.group(0))

            start_pos = match.start() + bias

            if link_name not in references_positions:

              references_positions[link_name] = [start_pos]

            else:

              references_positions[link_name].append(start_pos)

        text = re.sub(pattern, '', text)

      article_text += text + '\n'

      bias = len(article_text)

references_positions

In [ ]:
print(article_text[910:1000])

In [ ]:
links_by_sources = {}



for source_key, items in tqdm(links_by_citations.items()):

    extracted_links = []

    for item1 in items:

      item = item1[0]

      if item.startswith("http"):

        try:

          headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

          response = requests.get(item, headers=headers, stream=True, allow_redirects=False, timeout=(3, 7))

          response.raise_for_status()

          text = response.text

          if text:

            extracted_links.append(text)

        except requests.RequestException as e:

          continue

    if extracted_links:

      links_by_sources[source_key] = extracted_links

In [ ]:
texts_by_sources_traf = {}



for source_key, items in tqdm(links_by_sources.items()):

    extracted_texts = []

    for item in items:

      try:

        text = trafilatura.html2txt(item, clean=True)

        if text:

          extracted_texts.append(text)

      except requests.RequestException as e:

        continue

    if extracted_texts:

      texts_by_sources_traf[source_key] = extracted_texts

In [ ]:
for source, texts in texts_by_sources_traf.items():

    print(f"{source}:")

    for text in texts:

        print("  -", text[:200])

In [ ]:
texts_by_sources_news = {}



for source_key, items in tqdm(links_by_sources.items()):

    extracted_texts = []

    for item in items:

      try:

        text = fulltext(item)

        if text:

          extracted_texts.append(text)

      except:

        continue

    if extracted_texts:

      texts_by_sources_news[source_key] = extracted_texts

In [ ]:
for source, texts in texts_by_sources_news.items():

    for text in texts:

        print("  -", len(text), source, text[:200])

print(len(texts_by_sources_news.keys()))

In [ ]:
tmp2 = {}



for source_key, items in tqdm(links_by_citations.items()):

    extracted_links = []

    for item1 in items:

      item = item1[0]

      if item.startswith("http"):

        try:

          a = Article(item, language='ru')

          a.download()

          a.parse()

          text = a.text

          if text:

            extracted_links.append(text)

        except:

          continue

    if extracted_links:

      tmp2[source_key] = extracted_links

In [ ]:
current_length = 0
cleared_text = ""
flag = False
for text in article_text.split('\n\n'):
    current_length += len(text) 
    for link, num in references_positions.items():
        if num[0] <= current_length and num[0] >= current_length - len(text):
            if link not in tmp2:
                flag = True
                break
            else:
                break
    if flag:
        flag = False
        continue
    cleared_text += text + '\n'

In [ ]:
print(len(cleared_text), len(article_text))

In [ ]:
for key, value in cleared_content.items():

  print(key, '\n' + value)

print(len(cleared_content), len(content))

# STORM TEST

In [ ]:
!pip install openai

In [ ]:
from tqdm.asyncio import tqdm as async_tqdm

from functools import wraps

import asyncio

import logging

from tqdm.auto import tqdm

import re

import numpy as np

import wikipediaapi

from openai import AsyncOpenAI, OpenAI

In [ ]:
!pip install nest-asyncio

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
RELATED_TOPICS_PROMPT = """Я пишу статью Википедия на тему указанную ниже. Пожалуйста, порекомендуй несколько возможных страниц Википедия по смежным тематикам.

В первую очередь, меня интересуют страницы, которые могли бы более детально раскрыть наиболее интересные аспекты тесно связанные с темой,

а также страницы, которые помогли бы мне лучше понять типовое содержание и структуру статьи на подобные темы.

Пожалуйста, перечисли названия {} наиболее релевантных страниц используя формат "* PAGE NAME \\n"



Интересующая меня тема: {}"""



PERSPECTIVES_PROMPT = """Тебе надо отобрать группу редакторов для написания содержательнольной статьи Википедия на заданную тему.

Каждый редактор отражает конкретную точку зрения, роль или аффилиацию связанную с темой.

Для написания характеристик изучи оглавления страниц Википедия на связанные темы.

Для каждого редактора напиши на каких аспектах статьи он фокусируется.

Отвечай в следующему формате: "1. EDITOR NAME: SUMMARY DESCRIPTION \n 2. EDITOR NAME: SUMMARY DESCRIPTION \n ..."



Интересующая меня тема: {}

Оглавления страниц на связанную тему:

{}"""



QUESTIONS_PROMPT = """Ты опытный редактор статей Википедия и ты хочешь отредактировать конкретную страницу.

У тебя есть определенная направленность в методике проведения исследований темы.

Сейчас ты общаешься с экспертом по теме чтобы получить необходимую информацию для улучшения содержания страницы.

Задавай хорошие вопросы чтобы получить полезную информацию.

Как только у тебя закончились вопросы скажи "Больше вопросов нет. Большое спасибо за консультацию!" и закончи беседу.



Пожалуйста, задавай по одному вопросу за раз и не повторяй ранее озвученные вопросы.

Твои вопросы должны быть строго связаны с темой страницы, которую ты хочешь отредактировать.



Тема редактируемой страницы: {}

Твоя редакторская точка зрения: {}



Текущая история беседы:

{}

"""



QUERY_PROMPT = """Тебе нужно ответить на вопросы по теме используя поисковик Google.

Какие запросы нужно ввести в поисковую строку?

Напиши все необходимые запросы в формате: "* query 1\n * query 2\n..."



Обсуждаемая тема: {}

Вопрос на который нужно ответить: {}"""



ANSWER_PROMPT = """Ты эксперт, который отвечает на вопросы используя только предоставленные источники.

Ты общаешься с редактором Википедия, который хочет написать статью на известную тебе тему.

Ты уже собрал необходимые источники и сейчас используешь их для формирования достоверного ответа.

Постарайся сделать ответ как можно более информативным и убедись, что каждое написанное предложение

подтверждается собранными источниками.



Обсуждаемая тема: {}



Вопрос:

{}



Собранная информация:

{}

"""



COMPRESS_ANWER_PROMPT = """Ты эксперт, который отвечает на вопросы используя только предоставленные источники.

Ты общаешься с редактором Википедия, который хочет написать статью на известную тебе тему.

Ты уже собрал необходимые источники и сформировал полный ответ.

Теперь твоя задача написать короткий ответ в 2 предложениях.



Обсуждаемая тема: {}



Вопрос:

{}



Полный ответ:

{}

"""



NEARIDX_PROMPT = """Я работаю с поисковой системой на базе алгоритма BM25.

Запрос BM25 задается используя следующие операторы:



"A" AND "B" - оператор AND пересечения результатов по ключевому слову "A" с результатами по ключевому слову "B"

пример запроса:

"цк" AND "кпсс"



"A" OR "B" - оператор OR объединения результатов по ключевому слову "А" с результатами по ключевому слову "B"

пример запроса:

"рога" OR "копыта"



"A" NOT "B" - оператор NOT исключения результатов по ключевому слову "B" из результатов по ключевому слову "A"

пример запроса:

"долг" NOT ("чистый")



PREFIX* - поиск всех документов где есть фрагмент начинающийся с PREFIX

пример запроса:

электр*



Разбей запрос на ключевые слова и их синонимы, а затем используя правила BM25 подбери необходимую комбинацию операторов.

Покажи свои размышления используя следующие правила:

* все синонимы записывай в формате "phrase - [synonyms]"

* итоговое выражение запиши в формате "### ... ###"



Мой запрос:

{}

"""



CONVERSATION_WISE_OUTLINE_PROMPT = """На основе беседы с экспертом сочини короткий план страницы Википедия на заданную тему без конкретных деталей.

Вот формат в котором нужно все описать:

1. Начни план с названия интересующей темы "Topic"

2. Используй "# Title" для заголовков разделов, "## Title" для заголовков подразделов.

3. Ничего другого писать не нужно.



Интересующая меня тема: {}

История беседы с экспертом:

{}

"""



COMBINE_OUTLINE_PROMPT = """Тебе нужно сочинить короткий план для страницы Википедия.

У тебя уже есть несколько черновых вариантов, подготовленных экспертами по теме страницы.

Объедини эти варианты в общий короткий план, который раскроет основные темы статьи.

Вот формат в котором нужно все описать:

1. Начни план с названия интересующей темы "Topic"

2. Используй "# Title" для заголовков разделов, "## Title" для заголовков подразделов, "### Title" для подподразделов и так далее.

3. В конце плана напиши **END**

4. Ничего другого писать не нужно.



Интересующая меня тема: {}

Черновые варианты плана:

{}

"""



SECTION_WRITE_PROMPT = """Ты редактор Википедия. Тебе поручили написать одну из секций страницы Википедия на заданную тему.

Другой опытный редактор уже сочинил план секции и набрал нужные материал для написания секции.

Объедини этот материал по разделам плана соблюдая следующие правила:

1. Используй "# Title" для заголовков разделов, "## Title" для заголовков подразделов, "### Title" для подподразделов и так далее.

2. Используй только информацию, которая есть в собранном материале.

3. Текст должен быть подробными.

4. Все заголовки секции соответствовать плану секции.

5. Пожалуйста, пиши только на русском языке.



*Тема статьи:* {}

*Собранный материал:*

{}

*План секции:*

{}

"""





def print_sections(sections, level=0):

    res = []

    for s in sections:

        res.append("%s %s" % ("#"*(level + 1), s.title))

        res.extend(print_sections(s.sections, level + 1))

    return res





def get_outline(page):

    return "\n".join([page.title]+print_sections(page.sections))





def search_pages(wiki, query: str, limit: int = 3) -> list:

    """

    Searches for pages using a text query.

    :param query: The search query.

    :param limit: The maximum number of results to return. Default is 10.

    :return: A list of page titles.

    """

    params = {

        "action": "query",

        "list": "search",

        "srsearch": query,

        "srlimit": limit,

    }



    # Create a dummy page object for the _query method

    dummy_page = wikipediaapi.WikipediaPage(wiki, 0, language=wiki.language)

    data = wiki._query(dummy_page, params)

    # print(data)

    pageids = [result["title"] for result in data["query"]["search"]]



    return [wiki.page(pageid) for pageid in pageids]





def extract_perspective_descriptions(text):

    return re.findall(r"[0-9]+\. (.+)", text)[:-1]





def extract_queries(text):

    return re.findall(r"\* [\'\"]*(.+)[\'\"]*\n", text)





sentence_end_pattern = re.compile(r"(.+?)([\.\?\!]|\n\n)")





def trim_til_sentence_end(text):

    return sentence_end_pattern.sub(r"\2", text[::-1], count=1)[::-1].strip()





def uncapitalize(text):

    if text:

        return text[0].lower()+text[1:]

    return text





class OutlineNode:

    def __init__(self, text, parent=None):

        self.text = text.replace("#", "").strip()

        self.parent = parent

        self.level = text.count("#")

        self.children = []



    def add_child(self, node):

        self.children.append(node)

        node.parent = self



    def find_section(self, node):

        if node.level - 1 != self.level and len(self.children) > 0:

            self.children[-1].find_section(node)

        else:

            self.add_child(node)



    def simplify(self):

        for c in set(self.children):

            c.simplify()

        if self.parent is not None:

            if len(self.parent.children) < 2:

                for c in self.children:

                    c.level -= 1

                    self.parent.add_child(c)

                self.children = []

                self.parent.children.remove(self)



    def get_tuple_view(self, skip_level=0):

        res = []

        if self.level > skip_level:

            res.append((self.level, self.text))

        for c in self.children:

            res.extend(c.get_tuple_view(skip_level))

        return res



    def get_subtrees_texts(self, parent_prefix="", parent_level=1, skip_level=0):

        res = []

        if self.level > skip_level:

            res.append((parent_prefix, self.text))

        if self.level == parent_level:

            parent_prefix = self.text

        for c in self.children:

            res.extend(c.get_subtrees_texts(parent_prefix, parent_level, skip_level))

        return res



    def __repr__(self, indent=""):

        return (

            indent

            + f" {self.text}"

            + "\n"

            + "".join(child.__repr__(indent + "#") for child in self.children)

        )





def make_outline_tree(outline):

    outline_tree = None

    orig = outline.replace('\n\n', '\n').strip()

    for row in orig.split('\n'):

        row = OutlineNode(row)

        if outline_tree is None:

            outline_tree = row

        else:

            outline_tree.find_section(row)

    return outline_tree





def simplify_outline(outline):

    outline_tree = make_outline_tree(outline)

    outline_tree.simplify()

    return str(outline_tree)





def parse_and_normalize_outline(outline):

    outline = simplify_outline(outline)

    outline = outline.split('\n')

    outline = [o for o in outline if "####" not in o]

    return outline[1:]





def LOG_insert(file, format, text, level):

    infoLog = logging.FileHandler(file)

    infoLog.setFormatter(format)

    logger = logging.getLogger(file)

    logger.setLevel(level)



    if not logger.handlers:

        logger.addHandler(infoLog)

        if (level == logging.INFO):

            logger.info(text)

        if (level == logging.ERROR):

            logger.error(text)

        if (level == logging.WARNING):

            logger.warning(text)



    infoLog.close()

    logger.removeHandler(infoLog)



    return





def awaitify(sync_func):

    """Wrap a synchronous callable to allow ``await``'ing it"""

    @wraps(sync_func)

    async def async_func(*args, **kwargs):

        # print("task running")

        return sync_func(*args, **kwargs)

    return async_func





async def run_task_batch(afunc, arg_batch):

    if not asyncio.iscoroutinefunction(afunc):

        afunc = awaitify(afunc)

    tasks = []

    for args in arg_batch:

        t = asyncio.create_task(afunc(*args))

        tasks.append(t)



    results = await asyncio.gather(*tasks)

    # results = []

    # for task in tqdm(

    #     asyncio.as_completed(tasks),

    #     disable=True,

    #     total=len(tasks)

    # ):

    #     value = await task

    #     results.append(value)

    return results





def run_threadsafe(func, *args, **kwargs):



    async def async_func():

        return await func(*args, **kwargs)



    loop = asyncio.get_event_loop()

    return loop.run_until_complete(async_func())





def run_in_loop(coroutine):

    loop = asyncio.get_event_loop()

    completion = loop.run_until_complete(coroutine)

    return completion





class StormCaller:

    def __init__(self, open_ai_client, model_id=None, repetition_penalty=1.1) -> None:

        self.client = open_ai_client

        self.model_id = model_id

        self.backend_logger = None

        self.logger_file = None

        if model_id is None:

            tmp = run_threadsafe(self.client.models.list)

            self.model_id = tmp.data[0].id

        self.rep_pen = repetition_penalty



    @classmethod

    def name(cls):

        """Return name of class in lower case"""

        return cls.__name__.lower()



    @property

    def logger(self):

        if self.backend_logger is None:

            self.init_logger()

        return self.backend_logger



    def init_logger(self, file=None):

        if self.logger_file is not None:

            self.logger_file.close()

            self.backend_logger.removeHandler(self.logger_file)

            self.logger_file = None



        if file is not None:

            self.logger_file = logging.FileHandler(file, mode='w', encoding='utf-8')

            self.backend_logger = logging.getLogger(file)

        else:

            self.backend_logger = logging.getLogger(__name__ + '.' + self.name())

        self.backend_logger.handlers.clear()

        self.backend_logger.setLevel(logging.INFO)

        if self.logger_file is not None:

            ch = self.logger_file

        else:

            # handler_name = 'stream_handler.' + __name__ + '.' + self.name()

            # if handler_name in [h.name for h in self.backend_logger.handlers]:

            #     return

            ch = logging.StreamHandler()

            # ch.set_name(handler_name)

        ch.setLevel(logging.INFO)

        formatter = logging.Formatter('%(asctime)s: %(levelname)s: %(message)s')

        ch.setFormatter(formatter)

        self.backend_logger.addHandler(ch)



    async def get_completion(self, query, system=None, examples=None,

                             choices=None, rep_penalty=1.1,

                             regex_pattern=None, max_tokens=512):

        assert sum(map(lambda x: x is not None, [choices, regex_pattern])) < 2, "Only one guided mode is allowed"

        msgs = []

        if system is not None:

            msgs += [{"role": "system", "content": system}]

        if examples is not None:

            assert isinstance(examples, list)

            for q, a in examples:

                msgs += [{"role": "user", "content": q}]

                msgs += [{"role": "assistant", "content": a}]

        msgs += [{"role": "user", "content": query}]



        self.logger.info(f"PROMPT:\n{query}")

        completion = self.client.chat.completions.create(

            model=self.model_id,

            messages=msgs,

            temperature=0.01,

            top_p=0.9,

            max_tokens=max_tokens,

            extra_body={

                "repetition_penalty": rep_penalty,

                "guided_choice": choices,

                "add_generation_prompt": True,

                "guided_regex": regex_pattern

            }

        )

        # loop = asyncio.get_event_loop()

        # completion = loop.run_until_complete(completion)

        completion = await completion

        response = completion.choices[0].message.content.strip()

        self.logger.info(f"RESPONSE:\n{response}")

        return response



    def get_related_topics(self, topic, count=10):

        self.logger.info(f'Generating related topics for "{topic}"...')

        prompt = RELATED_TOPICS_PROMPT

        prompt = prompt.format(count, topic)

        # regex_pattern=r"(\* [^\*\n]+\n)"+"{{1,{}}}".format(count)))

        response = run_in_loop(self.get_completion(prompt,))

        # list(map(lambda x: x.removeprefix("* "), response.split('\n')))

        related_topics = re.findall("\* (.+?)\n", response)

        return related_topics



    def get_wiki_pages(self, topics, rank_func=None, top_k=None):

        self.logger.info('Fetching wiki pages for related topics...')

        wiki_wiki = wikipediaapi.Wikipedia('ecology survey', language='ru')

        found_pages = []

        for topic in topics:

            found_pages.extend(search_pages(wiki_wiki, topic))

        found_pages = {r.summary: r for r in found_pages}

        found_pages = list(found_pages.values())



        if top_k is None:

            top_k = len(topics)

        page_ids = rank_func(topic, found_pages, top_k)



        outlines = []

        for i in page_ids:

            page = found_pages[i]

            outlines.append(get_outline(page))



        return outlines



    def get_perspectives(self, topic, outlines):

        self.logger.info(f'Generating perspectives for "{topic}"...')

        prompt = PERSPECTIVES_PROMPT

        prompt = prompt.format(topic, "\n\n".join(outlines))

        res = run_in_loop(self.get_completion(prompt, max_tokens=1024))

        return extract_perspective_descriptions(res)



    async def get_questions(self, topic, perspective, conversation=[]):

        self.logger.info(f'Generating next question for perspective "{perspective}"...')

        prompt = QUESTIONS_PROMPT



        prompt = prompt.format(topic, perspective, "\n".join(conversation))

        # print(prompt)

        return await self.get_completion(prompt, max_tokens=256)



    async def get_queries(self, topic, question):

        self.logger.info('Converting question to queries...')

        prompt = QUERY_PROMPT



        prompt = prompt.format(topic, question)

        res = await self.get_completion(prompt, max_tokens=256)

        return extract_queries(res)



    async def get_bm25_query(self, query):

        self.logger.info('Converting query to bm25...')

        res = await self.get_completion(NEARIDX_PROMPT.format(query), max_tokens=1024)

        query = re.findall(r"###((?:\n|.)+?)###", res)

        if len(query) < 1:

            print(res)

        query = query[0].replace('\n', " ").replace("  ", " ").strip()

        return query



    async def get_answer(self, topic, question, sources):

        self.logger.info('Generating answer for question...')

        prompt = ANSWER_PROMPT



        prompt = prompt.format(topic, question, "\n\n".join(sources))

        return await self.get_completion(prompt, max_tokens=512, rep_penalty=self.rep_pen)



    async def compress_answer(self, topic, question, answer):

        self.logger.info('Compressing answer...')

        prompt = COMPRESS_ANWER_PROMPT



        prompt = prompt.format(topic, question, answer)

        return await self.get_completion(prompt, max_tokens=512)



    async def make_convo_outline(self, topic, conversation):

        self.logger.info('Converting conversation to outline...')

        prompt = CONVERSATION_WISE_OUTLINE_PROMPT



        prompt = prompt.format(topic, "\n".join(conversation))

        return await self.get_completion(prompt, max_tokens=1024)



    def combine_outlines(self, topic, outlines):

        self.logger.info('Combining outlines...')

        prompt = COMBINE_OUTLINE_PROMPT

        outlines = ["\n".join([f"Outline {i}"]+parse_and_normalize_outline(o)) for i, o in enumerate(outlines, start=1)]

        prompt = prompt.format(topic, "\n{}\n".format("*"*10).join(outlines))

        res = run_in_loop(self.get_completion(prompt, max_tokens=2048)).replace("**END**", "").strip()

        # print(res)

        return str(make_outline_tree(res))



    async def temporary_search():

      news_doc = tmp2

      texts_doc = []

      for doc in news_doc.values():

        texts_doc.append(doc)

      return texts_doc



    async def expand_perspective(

        self, cur_perspective, chosen_topic,

        document_search_func=temporary_search, is_bm25_search=True,

        question_limit=5

    ):

        references = []

        full_answers = []

        compressed_answers = []

        convo_hist = []



        for i in (tracker := tqdm(range(question_limit))):

            tracker.set_description("Generating question...")

            question = await self.get_questions(chosen_topic, cur_perspective, convo_hist)

# will need later

            # tracker.set_description("Searching for docs...")

            # queries = await self.get_queries(chosen_topic, question)



            # if is_bm25_search:

            #     tracker.set_description("Searching for docs... (converting queries to bm25)")

            #     res = await run_task_batch(self.get_bm25_query, ([q] for q in queries))

            #     queries = list(zip(queries, res))



            # tracker.set_description("Searching for docs... (fetching documents)")



            docs = set()

            # for q in queries:

            #     sr = await document_search_func(*q)

            #     docs.update(sr)

            found_d = await document_search_func()

            for d in found_d:

              references.append(d)



            tracker.set_description("Generating answer...")

            answer = await self.get_answer(chosen_topic, question, docs)

            answer = trim_til_sentence_end(answer)



            full_answers.append(answer)



            tracker.set_description("Compressing answer...")

            answer = await self.compress_answer(chosen_topic, question, answer)

            compressed_answers.append(answer)



            convo_hist.append("User:{}\nExpert:{}".format(question, answer))

        return references, full_answers, compressed_answers, convo_hist



    def get_section_documents(self, combined_outline, references, sts_emb, relevancy_percentile=85):

        # levels, outline_headings = list(zip(*make_outline_tree(combined_outline).get_tuple_view()))



        # outline_embs = sts_emb.encode(outline_headings, normalize_embeddings=True)

        # outline_embs *= np.array(levels).reshape(-1, 1)

        # outline_headings = list(map(lambda x: ", ".join(filter(None, x)),

        #                             make_outline_tree(combined_outline).get_subtrees_texts()))

        # outline_embs = sts_emb.encode(outline_headings, normalize_embeddings=True)



        refs = []

        for ref_group in tqdm(references):

            ref_group = [r for g in ref_group for r in g]

            refs.extend(ref_group)



        refs = list(set(refs))



        r_embs = sts_emb.encode(refs, normalize_embeddings=True)



        section_docs = []

        for section in make_outline_tree(combined_outline).children:

            section_text = ", ".join((uncapitalize(l.strip())

                                      for l in str(section).replace("#", "").split('\n'))).strip()

            section_emb = sts_emb.encode(section_text, normalize_embeddings=True)

            relevant_refs = r_embs @ section_emb.T

            mask = relevant_refs > np.percentile(relevant_refs, relevancy_percentile)

            order = np.argsort(-relevant_refs, axis=-1)

            # Trick to retain 0 indice

            relevant_refs = (order+1) * np.take_along_axis(mask, order, axis=-1)

            relevant_refs = relevant_refs[relevant_refs > 0] - 1

            section_docs.append((section.__repr__(indent='#'), relevant_refs))



        return section_docs, refs



    def write_wiki_section(self, topic, collected_docs, section_outline):

        prompt = SECTION_WRITE_PROMPT

        prompt = prompt.format(topic, "\n\n".join(collected_docs), section_outline)

        return self.get_completion(prompt, max_tokens=2048, rep_penalty=self.rep_pen)


In [ ]:
client = AsyncOpenAI(api_key='6a90f027a30e12cc1428a89775efdcfe19d1df8ad344567a0895b9d7b772ef05', base_url= "http://demo.labinform.ru:30101/v1")

In [ ]:
penalty = 1.0

storm_writter = StormCaller(client,  repetition_penalty=penalty)

In [ ]:
import numpy as np


def get_similarity(a, b, embedder):
    if isinstance(a, str):
        a = [a]
    if isinstance(b, str):
        b = [b]
    a = embedder.encode(a, normalize_embeddings=True)
    b = embedder.encode(b, normalize_embeddings=True)
    return a @ b.T


def filter_valid_pages(related_topics, pages, top_k, thresh=0.3):
    topic_similarity = get_similarity(
        related_topics, [p.summary for p in pages], sts_emb
    )
    topic_similarity = topic_similarity.max(axis=0)
    topic_similarity *= topic_similarity >= thresh
    return np.argsort(-topic_similarity)[:top_k]

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer

sts_model_name = "BAAI/bge-m3"

sts_emb = SentenceTransformer(sts_model_name, device="cuda")

In [ ]:
print(storm_writter.model_id)

In [ ]:
from pathlib import Path



log_dir = Path("storm_logs")

log_dir.mkdir(exist_ok=True)

In [ ]:
mod = 'qwen2.5-72b-instruct'



main_topic = "Летние олимпийские игры 2024"

# main_topic="уязвимость в процессорах Intel"

file_path = log_dir / "{}_{}_rp{}.log".format(

    main_topic.replace(" ", "-"), mod, penalty

)

storm_writter.init_logger(file_path.as_posix())

related_topics = storm_writter.get_related_topics(main_topic)

In [ ]:
wiki_pages = storm_writter.get_wiki_pages(related_topics, filter_valid_pages)

In [ ]:
perspectives = storm_writter.get_perspectives(main_topic, wiki_pages)

In [ ]:
perspectives

In [ ]:
references = []

full_answers = []

compressed_answers = []

conversations = []





tasks = []

for p in perspectives:

    t = storm_writter.expand_perspective(p, main_topic)

    tasks.append(t)



results = []

for task in tqdm(asyncio.as_completed(tasks), disable=True, total=len(tasks)):

    value = await task

    results.append(value)

# references.append(ref)

# full_answers.append(fa)

# compressed_answers.append(ca)

# conversations.append(conv)

In [ ]:
references = []

full_answers = []

compressed_answers = []

conversations = []



for ref, fa, ca, conv in results:

    references.append(ref)

    full_answers.append(fa)

    compressed_answers.append(ca)

    conversations.append(conv)

In [ ]:
convo_outlines = await run_task_batch(

    storm_writter.make_convo_outline, [[main_topic, c] for c in conversations]

)

In [ ]:
combined_outline = storm_writter.combine_outlines(main_topic, convo_outlines)

In [ ]:
section_docs, refs = storm_writter.get_section_documents(

    combined_outline, references, sts_emb

)

In [ ]:
from functools import reduce

from itertools import accumulate, takewhile



tok_limit = 32000

tok_p_wd = 3



max_wds = tok_limit // tok_p_wd



article = []

for section, docs in tqdm(section_docs):

    docs = [refs[d] for d in docs]

    thr = max(

        enumerate(

            takewhile(

                lambda x: x < max_wds,

                accumulate(map(lambda x: len(x.split()), docs), initial=0),

            )

        )

    )[0]

    thr = min(thr + 1, len(docs))

    docs = docs[:thr]

    section_text = storm_writter.write_wiki_section(main_topic, docs, section)

    article.append(section_text)



article = await asyncio.gather(*article)

In [ ]:
out_dir = "outputs"

out_dir = Path(out_dir)

out_dir.mkdir(exist_ok=True)



with open(

    out_dir / f"{main_topic.replace(' ', '-')}_wiki_{mod}_rp{penalty}.md",

    mode="w",

    encoding="utf-8",

) as f:

    f.write(

        f"# {main_topic.capitalize()}"

        + "\n\n".join([""] + article).replace("\n#", "\n##")

    )

In [ ]:
combined_outline

In [ ]:
tmp = np.random.random_sample((10, 20))

In [ ]:
print(len(combined_outline.split()))

In [ ]:
!pip install razdel

In [ ]:
from razdel import sentenize

In [ ]:
def get_sims(pred_text, ref_text, embedder):
    assert isinstance(pred_text, str) and isinstance(ref_text, str)
    
    # pred_sents = sentenize(pred_text)
    # ref_sents = sentenize(ref_text)
    # pred_sents_t = [p.text for p in pred_sents]
    # ref_sents_t = [r.text for r in ref_sents]
    pred_sents_t = [sent.text for par in pred_text.split('\n') for sent in sentenize(par) if sent.text.strip()]
    ref_sents_t = [sent.text for par in ref_text.split('\n') for sent in sentenize(par) if sent.text.strip()]
    # print(len(pred_sents_t), len(ref_sents_t))
    pred_emb = embedder.encode(pred_sents_t, normalize_embeddings=True)
    ref_emb =  embedder.encode(ref_sents_t, normalize_embeddings=True)
    sims = pred_emb @ ref_emb.T
    return sims, pred_sents_t, ref_sents_t

def get_bert_prec_rec(sims):
    emb_prec = sims.max(axis = 1).mean()
    emb_rec = sims.max(axis=0).mean()
    pred_ref_map = sims.argmax(axis=0).tolist()
    return emb_prec, emb_rec, pred_ref_map


In [ ]:
s, p, r = get_sims(combined_outline, article_text, labse_emb)

In [ ]:
print(s, p[:1], r[:1])

In [ ]:
pr, rec, m = get_bert_prec_rec(s)

In [ ]:
print(pr, rec, m)

In [ ]:
print(len(cleared_text), len(article_text))

In [ ]:
a = [sent.text for par in combined_outline.split('\n') for sent in sentenize(par) if sent.text.strip()]
print(a[4:8])

In [ ]:
0.5431373 0.5169309 [0, 25, 6, 6, 6, 6, 6, 1, 26, 6, 6, 6, 6, 29, 6, 6, 6, 6, 28, 48, 89, 89, 5, 96, 89, 104, 89, 29, 7, 101, 6, 20, 17, 79, 99, 29, 96, 100, 0, 0, 2, 5, 105, 48, 20, 20, 6, 0, 6, 89, 6, 100, 89, 5, 31, 8, 6, 5, 0, 28, 59, 56, 26, 7, 8, 99, 10, 19, 19, 19, 56, 56, 32, 29, 19, 31, 19, 12, 6, 59, 0, 0, 100, 0, 89, 28, 105, 98, 99, 6, 98, 98, 6, 56, 98, 98, 6, 5, 105, 22, 15, 105, 28, 29, 29, 99, 27, 99, 54, 30, 29, 31, 31, 31, 7, 6, 5, 99, 19, 0, 25, 29, 75, 6, 20, 19, 0, 6, 6, 81, 30, 0, 1, 90]

In [ ]:
labse = "sergeyzh/LaBSE-ru-turbo"

labse_emb = SentenceTransformer(labse, device="cuda")

In [ ]:
0.6293626 0.65214586 [22, 6, 5, 89, 5, 6, 1, 33, 6, 46, 5, 38, 99, 46, 96, 5, 105, 19, 96, 96, 96, 46, 46, 5, 5, 0, 46, 46, 46, 37, 7, 0, 99, 38, 19, 48, 48, 99, 28, 19, 12, 6, 19, 99, 89, 96, 89, 89, 25, 105, 105, 99, 6, 98, 98, 96, 56, 105, 98, 5, 5, 96, 38, 96, 105, 28, 0, 99, 99, 45, 30, 96, 31, 0, 2, 99, 46, 19, 59, 7, 1, 28]

In [ ]:
names = ["sergeyzh/LaBSE-ru-turbo", "BAAI/bge-m3", "deepvk/USER-base"]

for name in names:
    emb = SentenceTransformer(name, device="cuda")
    s, p, r = get_sims(combined_outline, article_text, emb)
    pr, rec, m = get_bert_prec_rec(s)
    print(name)
    print(pr)
    print(rec)
    print('\n\n')

In [ ]:
names = ["sergeyzh/LaBSE-ru-turbo", "BAAI/bge-m3", "deepvk/USER-base"]

for name in names:
    emb = SentenceTransformer(name, device="cuda")
    s, p, r = get_sims(combined_outline, cleared_text, emb)
    pr, rec, m = get_bert_prec_rec(s)
    print(name)
    print(pr)
    print(rec)
    print('\n\n')